# Docker-Python "Swarm Mode" Lab

Based on Mario's gist here: https://gist.github.com/l0rd/5186cc80f8f26dc7e9490abca4405830

# Requirements
- Docker >= 1.12 (docker-17.05-ce recommended)
- Docker machine
- Virtualbox

# Setup

## Install docker and docker-machine modules

## NOTE:
#### python-docker-machine installed from https://github.com/mjbright/python-docker-machine

Simple modification to depend upon 'docker' not 'docker-py'.

Installed using

'''python setup.py install'''


In [139]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

## Check that docker is installed, but not docker-py !!

In [138]:
!which pip
!pip list --format=columns | grep -i docker

!pip list --format=columns | grep docker-py || echo -e "\n\n**** ERROR: you have Docker-py installed!!"

/home/mjb/usr/anaconda3/bin/pip
docker                             2.2.1      
docker-pycreds                     0.2.1      
python-docker-machine              0.2.2      
docker-pycreds                     0.2.1      


In [140]:
import docker

EXPECT_DOCKER_VERSION='2.2.1'

print("docker module version=" + docker.version)

if docker.version != EXPECT_DOCKER_VERSION:
    print("\n\nERROR: Bad docker.py version: {} != expected {}".format(docker.version, EXPECT_DOCKER_VERSION))
  

docker module version=2.2.1


# Creating a Docker client connection to the Docker engine

During this lab we will use the installed Docker Client to talk to your Docker hosts.

Those hosts may be running 
- locally      controlled via docker-machine
- in the cloud controlled via docker-machine
- under Docker Toolbox controlled via docker-machine (https://docs.docker.com/machine/get-started/#create-a-machine)
- under Docker For Windows, controlled via docker-machine (https://docs.docker.com/docker-for-windows/#docker-settings)
- Docker for Mac, where you will only be able to have a single-node swarm (unless bursting out to cloud)

This notebook demonstrates the use of docker-machine to control the hosts.

#### NOTE: The following command is specific to my environment as I have a special DOCKER_HOST value set:

## Connection to a local Docker host

#### NOTE: This is for information, in our lab we will obtain a client connection via the python-docker-machine module, see below.

In [141]:
!. ~/.docker.rc; ! echo $DOCKER_HOST

Setting up alias/DOCKER_HOST for 17.05.0-ce [/home/mjb/DOCKER/docker-17.05.0-ce]
tcp://127.0.0.1:2475


#### NOTE: You may comment the 'DOCKER_HOST' lines below if you have no special configuration:

In [142]:
import docker, os

# Use your DOCKER_HOST value here:
DOCKER_HOST='tcp://127.0.0.1:2475'

os.environ['DOCKER_HOST']=DOCKER_HOST

# auto: use same api version as docker daemon
client = docker.from_env(version='auto') #, url='tcp://127.0.0.1:2475')

# Using python-docker-machine wrapper module

Here we will describe use of this Python module which provides Python access to the 'docker-machine' executable on your machine

### Referencing the docker-machine executable

In [143]:
import machine
import docker

m = machine.Machine(path="/usr/local/bin/docker-machine")

In [144]:
print(m)

### Cleanup:  Delete any existing machines

In [145]:
# If you want to delete existing machines:
# !docker-machine rm -f swmaster swnode1 swnode2

DELETE_MACHINES=True
if DELETE_MACHINES:
    for node in m.ls():
        if node['Name'] != '':
            print("Deleting node " + node['Name'])
            m.rm(machine=node['Name'],force=True)
            
m.ls()

Deleting node swmaster1
Deleting node swnode1
Deleting node swnode2


[{'Name': ''}]

### Creating a new machine

If creating a machine in some cloud provider you will need to first export some appropriate token or subscription id as
an appropriately named environment variable, e.g.
- DIGITALOCEAN_ACCESS_TOKEN for DigitalOcean
- AZURE_SUBSCRIPTION_ID for Microsoft Azure

Refer to https://docs.docker.com/machine/get-started-cloud/ for information,

and to https://docs.docker.com/machine/drivers/gce/ (select cloud provider in left-hand sidebar) for specifics.

**WARNING**: You may need to "logon" to the platform before using the python-docker-machine otherwise the python-docker-machine module will block asking you to validate - but **you will not see that message** !

## Examples:

### Digital Ocean:  
Set DIGITALOCEAN_ACCESS_TOKEN as an environment variable 

e.g.
```
     os.environ['DIGITALOCEAN_ACCESS_TOKEN']='MY-TOKEN'
     m.create('swmaster1', driver='digitalocean', blocking=True)
```

### Azure:
Set AZURE_SUBSCRIPTION_ID as an environment variable 

e.g.
```
    os.environ['AZURE_SUBSCRIPTION_ID']='MY-ID'
    m.create('swmaster1', driver='azure', blocking=True)
```

### Google Compute Engine:
Use "gcloud auth login"

e.g.
```
     m.create('swmaster1', driver='google', blocking=True)
```

## Example of local machine creation

Below is an example of machine creation on a local machine, to be adapted as above to run in the cloud

In [146]:
m.create('test-machine', driver='virtualbox', blocking=True)

m.ls()

[{'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'test-machine',
  'ResponseTime': '274ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.100:2376'},
 {'Name': ''}]

## Deletion of a local machine

In [147]:
m.rm(machine='test-machine')

m.ls()

[{'Name': ''}]

# Create nodes for your swarm cluster (1 master and 2 worker nodes)

We will create 3 nodes using docker-machine/virtualbox.

In [148]:
#### If not already created, we can create our machines here:

import datetime
import time
import os

# MACHINE_DRIVER='virtualbox'

# FAILS on AZURE due to interactive validation required !
#MACHINE_DRIVER='azure'
#os.environ['AZURE_SUBSCRIPTION_ID']='azure-sub-id'

MACHINE_DRIVER='digitalocean'
#os.environ['DIGITALOCEAN_ACCESS_TOKEN']='my-token'

#MASTER_NODES=['swmaster1']
#WORKER_NODES=['swnode1','swnode2']

MASTER_NODES=['swmaster1','swmaster2','swmaster3']
WORKER_NODES=['swnode1','swnode2']

In [149]:
def createNodes(nodes):
    for node in nodes:
        NOW=datetime.datetime.now().strftime("%H:%M:%S")
        SECS_BEFORE=int(time.time())
        print("{} Creating node {} <on {}> ...".format(NOW, node, MACHINE_DRIVER))
        m.create(node, driver=MACHINE_DRIVER, blocking=False)
        SECS_AFTER=int(time.time())
        print("... took {} secs".format(SECS_AFTER-SECS_BEFORE))

    
CREATE_MACHINES=True
if CREATE_MACHINES:
    createNodes(MASTER_NODES + WORKER_NODES)

21:03:21 Creating node swmaster1 <on digitalocean> ...
... took 135 secs
21:05:36 Creating node swmaster2 <on digitalocean> ...
... took 113 secs
21:07:29 Creating node swmaster3 <on digitalocean> ...
... took 139 secs
21:09:48 Creating node swnode1 <on digitalocean> ...
... took 113 secs
21:11:41 Creating node swnode2 <on digitalocean> ...
... took 139 secs


#### NOTE:

From the command-line you can see the progress of machine creation, using
    ```$ docker-machine ls```
    
You may see errors during creation of machines before ssh connectivity is established and before docker host is started.

In [150]:
m.ls()

[{'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'digitalocean',
  'Error': '',
  'Name': 'swmaster1',
  'ResponseTime': '2.102s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://104.236.74.87:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'digitalocean',
  'Error': '',
  'Name': 'swmaster2',
  'ResponseTime': '2.302s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://104.131.173.175:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'digitalocean',
  'Error': '',
  'Name': 'swmaster3',
  'ResponseTime': '2.375s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://104.236.221.68:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'digitalocean',
  'Error': '',
  'Name': 'swnode1',
  

We can obtain the "config" parameters (o/p of "docker-machine config <machine name>")

In [151]:
m.config(machine='swmaster1')

{'base_url': 'https://104.236.74.87:2376',
 'tls': <docker.tls.TLSConfig at 0x7f316a4cdb00>}

We can obtain the "env" parameters (o/p of "docker-machine env <machine name>")

In [152]:
m.env(machine='swmaster1')

['export',
 'DOCKER_TLS_VERIFY="1"',
 'export',
 'DOCKER_HOST="tcp://104.236.74.87:2376"',
 'export',
 'DOCKER_CERT_PATH="/home/mjb/.docker/machine/machines/swmaster1"',
 'export',
 'DOCKER_MACHINE_NAME="swmaster1"',
 '#',
 'Run',
 'this',
 'command',
 'to',
 'configure',
 'your',
 'shell:',
 '#',
 'eval',
 '$(/usr/local/bin/docker-machine',
 'env',
 'swmaster1)']

Now let's create docker-py client objects for each of our new nodes

In [153]:
# Create Docker clients for each machine:

nodes={}

swmaster1 = docker.DockerClient(**m.config(machine='swmaster1'))
nodes['swmaster1']=swmaster1

swmaster2 = docker.DockerClient(**m.config(machine='swmaster2'))
nodes['swmaster2']=swmaster2

swmaster3 = docker.DockerClient(**m.config(machine='swmaster3'))
nodes['swmaster3']=swmaster3

swnode1 = docker.DockerClient(**m.config(machine='swnode1'))
nodes['swnode1']=swnode1

swnode2 = docker.DockerClient(**m.config(machine='swnode2'))
nodes['swnode2']=swnode2

pp.pprint(nodes)

{   'swmaster1': <docker.client.DockerClient object at 0x7f316a4cdc50>,
    'swmaster2': <docker.client.DockerClient object at 0x7f316a4cd048>,
    'swmaster3': <docker.client.DockerClient object at 0x7f316a4bc0b8>,
    'swnode1': <docker.client.DockerClient object at 0x7f316a4bc630>,
    'swnode2': <docker.client.DockerClient object at 0x7f316a879e10>}


In [155]:
# Check connectivity to machines:

print("Checking that nodes are 'ping'able")

#for node in MASTER_NODES + WORKER_NODES:
for node in nodes.keys():
    #print(node)
    print("{}: {}".format(node, nodes[node].ping()))

#print(swmaster.ping(), swnode1.ping(), swnode2.ping())

Checking that nodes are 'ping'able
swmaster1: True
swmaster2: True
swmaster3: True
swnode1: True
swnode2: True


In [156]:
swmaster1.images.list()

[]

# swarm init

Now that we have 3 nodes available, we will initialize our Swarm Cluster with 1 master node.


Including these parameters on the docker command line will connect the client to the docker daemon running on node '*swmaster*'.

### Networks before creation of swarm cluster
Before going further let's look at the networks on your machine.

Later, we'll see how a new network is created once the swarm cluster has been created.

In [157]:

def show_networks(client):
    FORMAT_STR="%-14s %-20s %-10s %-14s"
    print(FORMAT_STR % ("NETWORK ID", "NAME", "DRIVER", "SCOPE"))

    for network in client.networks.list():
        #print(dir(network))
        print(FORMAT_STR % (network.short_id, network.name, network.attrs['Driver'], network.attrs['Scope']))
        
        
# NETWORK ID          NAME                DRIVER              SCOPE
# ab973c7b9062        bridge              bridge              local
# e953fc8fe3f2        host                host                local
# d45eb5315d7c        none                null                local

show_networks(swmaster1)

NETWORK ID     NAME                 DRIVER     SCOPE         
1e5a827ec8     host                 host       local         
90e461294d     bridge               bridge     local         
fa6531249b     none                 null       local         


### Obtain ip addresses of nodes, esp. the 1st master node

Now let's identify the ip address of our nodes.

We need to note the ip address of the master node, so that when we initialize our swarm cluster, this node will advertise itself on this address so that other nodes can join the swarm cluster.

We can see this through config or ip commands of docker-machine as shown below.

In [158]:

for node in nodes.keys():
    ip = m.ip(machine=node)
    print("{}: {}".format(node, ip))

    
master1ip = m.ip(machine='swmaster1')

swmaster1: 104.236.74.87
swmaster2: 104.131.173.175
swmaster3: 104.236.221.68
swnode1: 104.236.29.189
swnode2: 104.131.74.237


We could then provide the above ip address as parameter to --advertise-addr when initializing the swarm.

However, it is quite convenient to run the above commands embedded, as below, as arguments to the swarm init command.

docker-machine config swmaster provides the parameters to use when connecting to the appropriate docker engine for our machine "swmaster".

The following command will run swarm init to generate the cluster with 'swmaster' as the Master node.
You should see output similar to the below:

In [159]:
swmaster1.info()

{'Architecture': 'x86_64',
 'BridgeNfIp6tables': True,
 'BridgeNfIptables': True,
 'CPUSet': True,
 'CPUShares': True,
 'CgroupDriver': 'cgroupfs',
 'ClusterAdvertise': '',
 'ClusterStore': '',
 'ContainerdCommit': {'Expected': '9048e5e50717ea4497b757314bad98ea3763c145',
  'ID': '9048e5e50717ea4497b757314bad98ea3763c145'},
 'Containers': 0,
 'ContainersPaused': 0,
 'ContainersRunning': 0,
 'ContainersStopped': 0,
 'CpuCfsPeriod': True,
 'CpuCfsQuota': True,
 'Debug': False,
 'DefaultRuntime': 'runc',
 'DockerRootDir': '/var/lib/docker',
 'Driver': 'aufs',
 'DriverStatus': [['Root Dir', '/var/lib/docker/aufs'],
  ['Backing Filesystem', 'extfs'],
  ['Dirs', '0'],
  ['Dirperm1 Supported', 'true']],
 'ExperimentalBuild': False,
 'HttpProxy': '',
 'HttpsProxy': '',
 'ID': '54LC:R7IH:2B2R:HOUM:IGYM:OSTZ:6NIG:ZP4J:MX6E:JLXM:VCYF:7YR3',
 'IPv4Forwarding': True,
 'Images': 0,
 'IndexServerAddress': 'https://index.docker.io/v1/',
 'InitBinary': 'docker-init',
 'InitCommit': {'Expected': '949e6fa

In [160]:
swmaster1.info()['Swarm']

{'ControlAvailable': False,
 'Error': '',
 'LocalNodeState': 'inactive',
 'NodeAddr': '',
 'NodeID': '',
 'RemoteManagers': None}

### Before creating the swarm cluster we have 0 nodes, and an inactive cluster

In [161]:
if 'Nodes' in swmaster1.info()['Swarm']:
    swmaster1.info()['Swarm']['Nodes']

In [162]:
swmaster1.info()['Swarm']['LocalNodeState']

'inactive'

In [163]:
swmaster1.swarm.init?

In [164]:
swmaster1.swarm.init(advertise_addr=master1ip)

In [165]:
swarm_info = swmaster1.swarm.client.info()

pp.pprint(swarm_info)

{   'Architecture': 'x86_64',
    'BridgeNfIp6tables': True,
    'BridgeNfIptables': True,
    'CPUSet': True,
    'CPUShares': True,
    'CgroupDriver': 'cgroupfs',
    'ClusterAdvertise': '',
    'ClusterStore': '',
    'ContainerdCommit': {   'Expected': '9048e5e50717ea4497b757314bad98ea3763c145',
                            'ID': '9048e5e50717ea4497b757314bad98ea3763c145'},
    'Containers': 0,
    'ContainersPaused': 0,
    'ContainersRunning': 0,
    'ContainersStopped': 0,
    'CpuCfsPeriod': True,
    'CpuCfsQuota': True,
    'Debug': False,
    'DefaultRuntime': 'runc',
    'DockerRootDir': '/var/lib/docker',
    'Driver': 'aufs',
    'DriverStatus': [   ['Root Dir', '/var/lib/docker/aufs'],
                        ['Backing Filesystem', 'extfs'],
                        ['Dirs', '0'],
                        ['Dirperm1 Supported', 'true']],
    'ExperimentalBuild': False,
    'HttpProxy': '',
    'HttpsProxy': '',
    'ID': '54LC:R7IH:2B2R:HOUM:IGYM:OSTZ:6NIG:ZP4J:MX6E:JLXM:V

### After creating the swarm cluster we have 1 nodes, and an active cluster

In [166]:
swmaster1.info()['Swarm']['Nodes']

1

In [167]:
swmaster1.info()['Swarm']['LocalNodeState']

'active'

In [168]:
pp.pprint(swmaster1.swarm.attrs)

{   'CreatedAt': '2017-05-08T19:22:37.568616358Z',
    'ID': '6ki5b9ozkew372ffiu72g7hp0',
    'JoinTokens': {   'Manager': 'SWMTKN-1-1x74ye5k0foltrh8xloj7rkfny92iypebym859djza6t0xbgvz-cicn8a0efsrmqzf98mdfbreuj',
                      'Worker': 'SWMTKN-1-1x74ye5k0foltrh8xloj7rkfny92iypebym859djza6t0xbgvz-16bbry01f31hf7xmrmvr3bh7y'},
    'Spec': {   'CAConfig': {'NodeCertExpiry': 7776000000000000},
                'Dispatcher': {'HeartbeatPeriod': 5000000000},
                'EncryptionConfig': {'AutoLockManagers': False},
                'Labels': {},
                'Name': 'default',
                'Orchestration': {'TaskHistoryRetentionLimit': 5},
                'Raft': {   'ElectionTick': 3,
                            'HeartbeatTick': 1,
                            'KeepOldSnapshots': 0,
                            'LogEntriesForSlowFollowers': 500,
                            'SnapshotInterval': 10000},
                'TaskDefaults': {}},
    'UpdatedAt': '2017-05-08T19:22:37.

### We can obtain the join tokens allowing to join new nodes as manager or worker nodes

In [169]:
join_tokens = swmaster1.swarm.attrs['JoinTokens']

pp.pprint(join_tokens)

manager_token = join_tokens['Manager']
worker_token = join_tokens['Worker']

{   'Manager': 'SWMTKN-1-1x74ye5k0foltrh8xloj7rkfny92iypebym859djza6t0xbgvz-cicn8a0efsrmqzf98mdfbreuj',
    'Worker': 'SWMTKN-1-1x74ye5k0foltrh8xloj7rkfny92iypebym859djza6t0xbgvz-16bbry01f31hf7xmrmvr3bh7y'}


A docker info should now show "Swarm: active" as below:

In [171]:
pp.pprint(swarm_info)

{   'Architecture': 'x86_64',
    'BridgeNfIp6tables': True,
    'BridgeNfIptables': True,
    'CPUSet': True,
    'CPUShares': True,
    'CgroupDriver': 'cgroupfs',
    'ClusterAdvertise': '',
    'ClusterStore': '',
    'ContainerdCommit': {   'Expected': '9048e5e50717ea4497b757314bad98ea3763c145',
                            'ID': '9048e5e50717ea4497b757314bad98ea3763c145'},
    'Containers': 0,
    'ContainersPaused': 0,
    'ContainersRunning': 0,
    'ContainersStopped': 0,
    'CpuCfsPeriod': True,
    'CpuCfsQuota': True,
    'Debug': False,
    'DefaultRuntime': 'runc',
    'DockerRootDir': '/var/lib/docker',
    'Driver': 'aufs',
    'DriverStatus': [   ['Root Dir', '/var/lib/docker/aufs'],
                        ['Backing Filesystem', 'extfs'],
                        ['Dirs', '0'],
                        ['Dirperm1 Supported', 'true']],
    'ExperimentalBuild': False,
    'HttpProxy': '',
    'HttpsProxy': '',
    'ID': '54LC:R7IH:2B2R:HOUM:IGYM:OSTZ:6NIG:ZP4J:MX6E:JLXM:V

If we look at the networks we should now see new networks such as '*ingress*' an overlay network and docker_gwbridge for the swarm cluster.

In [172]:
show_networks(swmaster1)
#show_networks(swmaster2)

NETWORK ID     NAME                 DRIVER     SCOPE         
0pp3luxc8r     ingress              overlay    swarm         
1e5a827ec8     host                 host       local         
90e461294d     bridge               bridge     local         
fa6531249b     none                 null       local         
1510d57c25     docker_gwbridge      bridge     local         


# swarm join

Now we wish to join Master and Worker nodes to our swarm cluster, to do this we need to obtain the token generated during the "swarm init".

In [173]:
print(manager_token)

SWMTKN-1-1x74ye5k0foltrh8xloj7rkfny92iypebym859djza6t0xbgvz-cicn8a0efsrmqzf98mdfbreuj


In [174]:
pp.pprint(swmaster1.info()['Swarm'])

{   'Cluster': {   'CreatedAt': '2017-05-08T19:22:37.568616358Z',
                   'ID': '6ki5b9ozkew372ffiu72g7hp0',
                   'Spec': {   'CAConfig': {'NodeCertExpiry': 7776000000000000},
                               'Dispatcher': {'HeartbeatPeriod': 5000000000},
                               'EncryptionConfig': {'AutoLockManagers': False},
                               'Labels': {},
                               'Name': 'default',
                               'Orchestration': {   'TaskHistoryRetentionLimit': 5},
                               'Raft': {   'ElectionTick': 3,
                                           'HeartbeatTick': 1,
                                           'KeepOldSnapshots': 0,
                                           'LogEntriesForSlowFollowers': 500,
                                           'SnapshotInterval': 10000},
                               'TaskDefaults': {}},
                   'UpdatedAt': '2017-05-08T19:22:37.72028698Z',
    

In [175]:
print(master1ip)

104.236.74.87


## Add 2nd, 2rd master nodes if present

In [176]:
if swmaster2 != None:
    swmaster2.swarm.join(join_token=manager_token, remote_addrs=[master1ip+':2377'],
                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

In [177]:
if swmaster3 != None:
    swmaster3.swarm.join(join_token=manager_token, remote_addrs=[master1ip+':2377'],
                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

In [178]:
!. ~/.docker.rc; docker $(docker-machine config swmaster1) node list

Setting up alias/DOCKER_HOST for 17.05.0-ce [/home/mjb/DOCKER/docker-17.05.0-ce]
ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
fuchgyw4eg2kax1cgqzd8vzzh     swmaster3           Ready               Active              Reachable
ttx2b261g5lwi2j7fntrr3tdl     swmaster2           Ready               Active              Reachable
xqujlss033v27mu6loigt59oc *   swmaster1           Ready               Active              Leader


In [179]:
def show_nodes(client):
    FORMAT_STR="%-14s %-10s %-10s %-14s %-10s"
    print(FORMAT_STR % ("ID", "HOSTNAME", "STATE", "AVAILABILITY", "MANAGER STATUS"))
    
    for node in client.nodes.list():
        #pp.pprint(node.attrs)
        nodeSwarmStatus=''
        state = node.attrs['Status']['State']
        if 'ManagerStatus' in node.attrs:
            if 'Leader' in node.attrs['ManagerStatus'] and node.attrs['ManagerStatus']['Leader']:
                nodeSwarmStatus='leader'
            elif 'Reachability' in node.attrs['ManagerStatus']:
                nodeSwarmStatus=node.attrs['ManagerStatus']['Reachability']
        print(FORMAT_STR % (node.id[:12], node.attrs['Description']['Hostname'], state, node.attrs['Spec']['Availability'], nodeSwarmStatus))

        
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
fuchgyw4eg2k   swmaster3  ready      active         reachable 
ttx2b261g5lw   swmaster2  ready      active         reachable 
xqujlss033v2   swmaster1  ready      active         leader    


## Add worker nodes

In [180]:
print(worker_token)

SWMTKN-1-1x74ye5k0foltrh8xloj7rkfny92iypebym859djza6t0xbgvz-16bbry01f31hf7xmrmvr3bh7y


Now we can use this token to join nodes as a worker to this cluster

Note: we could also join nodes as Master, but we have only 3 nodes available.

Let's join swnode1 as a worker node

In [181]:
#print( "swmaster.nodes()=" + str( swmaster.nodes() ))
#print()

import json

json_nodes = swmaster1.nodes.list()

#print(type(json_nodes))

for node in json_nodes:
    pp.pprint("node {}.attrs=".format(node.attrs['Description']['Hostname']))
    pp.pprint(node.attrs)
    print()
    

'node swmaster3.attrs='
{   'CreatedAt': '2017-05-08T19:26:12.803403204Z',
    'Description': {   'Engine': {   'EngineVersion': '17.05.0-ce',
                                     'Labels': {'provider': 'digitalocean'},
                                     'Plugins': [   {   'Name': 'bridge',
                                                        'Type': 'Network'},
                                                    {   'Name': 'host',
                                                        'Type': 'Network'},
                                                    {   'Name': 'macvlan',
                                                        'Type': 'Network'},
                                                    {   'Name': 'null',
                                                        'Type': 'Network'},
                                                    {   'Name': 'overlay',
                                                        'Type': 'Network'},
                                

In [182]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
fuchgyw4eg2k   swmaster3  ready      active         reachable 
ttx2b261g5lw   swmaster2  ready      active         reachable 
xqujlss033v2   swmaster1  ready      active         leader    


In [183]:
swmaster1.nodes.list()

[<Node: fuchgyw4eg>, <Node: ttx2b261g5>, <Node: xqujlss033>]

In [184]:
swnode1.swarm.join?

In [185]:
swnode1.swarm.join(join_token=worker_token, remote_addrs=[master1ip+':2377'],
                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

True

In [186]:
swmaster1.nodes.list()

[<Node: fuchgyw4eg>,
 <Node: h3vft22hw8>,
 <Node: ttx2b261g5>,
 <Node: xqujlss033>]

In [187]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
fuchgyw4eg2k   swmaster3  ready      active         reachable 
h3vft22hw8za   swnode1    ready      active                   
ttx2b261g5lw   swmaster2  ready      active         reachable 
xqujlss033v2   swmaster1  ready      active         leader    


In [188]:
swnode2.swarm.join(join_token=worker_token, remote_addrs=[master1ip+':2377'],
                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

True

In [189]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
fuchgyw4eg2k   swmaster3  ready      active         reachable 
h3vft22hw8za   swnode1    ready      active                   
i9126qign5e5   swnode2    ready      active                   
ttx2b261g5lw   swmaster2  ready      active         reachable 
xqujlss033v2   swmaster1  ready      active         leader    


### Need to force 'leave' to remove managers from cluster

## NOTE: dont remove swarm master nodes, this seems to break cluster ... currently

In [190]:
for node in swmaster1.nodes.list():
    pp.pprint(node.attrs['Status'])

{'Addr': '104.236.221.68', 'State': 'ready'}
{'Addr': '104.236.29.189', 'State': 'ready'}
{'Addr': '104.131.74.237', 'State': 'ready'}
{'Addr': '104.131.173.175', 'State': 'ready'}
{'Addr': '104.236.74.87', 'State': 'ready'}


In [107]:
# Just for info - DON'T DO THIS (will prevent service creation from working - bug)

# swmaster2.swarm.leave(force=True)
# swmaster3.swarm.leave(force=True)

True

In [108]:
show_nodes(swmaster1)

ID             HOSTNAME   STATUS     AVAILABILITY   MANAGER STATUS
14es9c9bhcym   swmaster2  down       active         unreachable
7sm2ij3pgadu   swmaster1  ready      active         leader    
9u7l4zrewb5h   swnode2    ready      active                   
wse9c8itne7y   swnode1    ready      active                   
zotk91il0kuc   swmaster3  ready      active         unreachable


In [109]:
# Just for info - DON'T DO THIS (will prevent service creation from working - bug)

# m.rm(machine='swmaster2',force=True)
# m.rm(machine='swmaster3',force=True)

True

In [115]:
m.rm(machine='swmaster3',force=True)

True

In [127]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
14es9c9bhcym   swmaster2  down       active         unreachable
7sm2ij3pgadu   swmaster1  ready      active         leader    
9u7l4zrewb5h   swnode2    ready      active                   
wse9c8itne7y   swnode1    ready      active                   
zotk91il0kuc   swmaster3  ready      active         unreachable


### NOTE: incorrect status

Our show_nodes method shows the managers are unreachable but still active and not necessarily down.

See below, this seems to be a problem in the API, or docker module implementation:

In [130]:
for node in swmaster1.nodes.list():
    pp.pprint(node.attrs['Status'])

{'Addr': '104.131.82.69', 'Message': 'heartbeat failure', 'State': 'down'}
{'Addr': '104.131.42.152', 'State': 'ready'}
{'Addr': '104.131.60.217', 'State': 'ready'}
{'Addr': '104.131.36.193', 'State': 'ready'}
{'Addr': '104.131.82.251', 'State': 'ready'}


# start service

First we check for any running services - there should be none in our newly initialized cluster:

In [191]:
swmaster1.services.list()

[]

Now we will create a new service based on the docker image mariolet/docker-demo

We will expose this service on port 8080


In [192]:
swmaster1.services.create??

In [193]:
docker.types.ContainerSpec?

## Create a service

Now let us use the Client API to create a new service

Note how we create ContainerSpec and TaskTemplate types wich we pass to the Client.Services.create() method

Here is a first version, which launches an alpine image which runs for 60 seconds:

In [135]:
IMAGE = 'alpine'

container_spec = docker.types.ContainerSpec(
    IMAGE, ['echo', 'hello']
)

"""
endpoint_spec = docker.types.EndpointSpec(ports={
    12357: (1990, 'udp'),
    12562: (678,),
    53243: 8080,
})
"""
endpoint_spec = docker.types.EndpointSpec(ports={
    8080: 8080,
})

task_tmpl = docker.types.TaskTemplate(container_spec)

In [194]:
swmaster1.services.create??

In [195]:
#swmaster1.services.create(task_tmpl, name='docker-demo', endpoint_spec=endpoint_spec)

service = swmaster1.services.create(
            # create arguments
            name='alpine-test',
            labels={'foo': 'bar'},
            # ContainerSpec arguments
            image=IMAGE,
            command="sleep 60",
            container_labels={'container': 'label'}
        )

In [196]:
swmaster1.services.list()

[<Service: tsvu8mr156>]

after 1 min ...

In [197]:
swmaster1.services.list()

[<Service: tsvu8mr156>]

In [1]:
service = swmaster1.services.list()[0]
print(service)
service_id = service.id

print(service.attrs)

for task in service.tasks():
    pp.pprint(task)
    break
    #exit()
#pp.pprint(service.tasks())


NameError: name 'swmaster1' is not defined

In [94]:
swmaster1.containers.list()

[]

In [ ]:
#swmaster1.services.create(image=IMAGE, name='docker-demo', labels={'label1': 'label2'}, command=None)

In [ ]:
NOW LAUNCH docker-demo

In [97]:
IMAGE = 'mariolet/docker-demo:20'
IMAGE = 'mjbright/docker-demo:20'

container_spec = docker.types.ContainerSpec(
    IMAGE, ['echo', 'hello']
)

endpoint_spec = docker.types.EndpointSpec(ports={
    8080: 8080,
})


task_tmpl = docker.types.TaskTemplate(container_spec)

docker_demo_service = swmaster1.services.create(
            # create arguments
            name='docker-demo',
            labels={'label1': 'value1'},
            # ContainerSpec arguments
            image=IMAGE,
            #command="sleep 60",
            container_labels={'container': 'label'},
    endpoint_spec=endpoint_spec
        )


In [100]:
#BUT PORTS NOT SHOWING:
    
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service ps docker-demo

Setting up alias/DOCKER_HOST for 17.03.0-ce [/home/mjb/DOCKER/docker-17.03.0-ce]
ID            NAME           IMAGE                    NODE     DESIRED STATE  CURRENT STATE          ERROR  PORTS
xih50ozcc6bq  docker-demo.1  mjbright/docker-demo:20  swnode1  Running        Running 2 minutes ago         


In [107]:
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service create --replicas 1 --name docker-demo-via-cli -p 9090:8080 mariolet/docker-demo:20

Setting up alias/DOCKER_HOST for 17.03.0-ce [/home/mjb/DOCKER/docker-17.03.0-ce]
bk8zos222khy87rzqxbha1qc4


In [ ]:
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service ps docker-demo-via-cli
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service rm docker-demo-via-cli

Now we list services again and we should see our newly added docker-demo service

In [113]:
swmaster1.services.list()

[<Service: byxykanzxx>, <Service: nlyczl6x63>]

In [121]:
def showServices(master):
    for service in master.services.list():
        print("{}: {}".format(service, service.name))

showServices(swmaster1)

<Service: byxykanzxx>: testname
<Service: nlyczl6x63>: docker-demo


In [126]:
swmaster1.services.get(service_id='byxykanzxx').remove()

showServices(swmaster1)

<Service: nlyczl6x63>: docker-demo


... and we can look at the service as seen by the cluster:

In [ ]:

docker $(docker-machine config swmaster1) service ps docker-demo

... and we can look at the service on the individual cluster nodes.

Of course as we set replicas to 1 there is only 1 replica of the service for the moment, running on just 1 node of our cluster:

In [128]:
service = swmaster1.services.list()[0]

service.tasks()


[{'CreatedAt': '2017-05-03T21:14:52.675436545Z',
  'DesiredState': 'running',
  'ID': 'xih50ozcc6bq663kpubk4aqby',
  'Labels': {},
  'NetworksAttachments': [{'Addresses': ['10.255.0.10/16'],
    'Network': {'CreatedAt': '2017-05-03T20:47:10.978500208Z',
     'DriverState': {'Name': 'overlay',
      'Options': {'com.docker.network.driver.overlay.vxlanid_list': '4096'}},
     'ID': 'b2nkmd57pjunz0n12f7rofdra',
     'IPAMOptions': {'Configs': [{'Gateway': '10.255.0.1',
        'Subnet': '10.255.0.0/16'}],
      'Driver': {'Name': 'default'}},
     'Spec': {'DriverConfiguration': {},
      'IPAMOptions': {'Configs': [{'Gateway': '10.255.0.1',
         'Subnet': '10.255.0.0/16'}],
       'Driver': {}},
      'Labels': {'com.docker.swarm.internal': 'true'},
      'Name': 'ingress'},
     'UpdatedAt': '2017-05-03T20:47:10.980143313Z',
     'Version': {'Index': 7}}}],
  'NodeID': 'n9xctssozkkuszohp7ikngdv9',
  'ServiceID': 'nlyczl6x63dlwqk6iss7wsz7h',
  'Slot': 1,
  'Spec': {'ContainerSpec': {

In [ ]:
service = swnode1.services.list()[0]

service.tasks()

In [130]:
service = swnode2.services.list()[0]

service.tasks()

ReadTimeout: HTTPSConnectionPool(host='192.168.99.103', port=2376): Read timed out. (read timeout=60)

In [ ]:
docker $(docker-machine config swnode1) ps

In [ ]:
docker $(docker-machine config swnode2) ps

# Accessing the service

As we are working remotely we need to create an ssh tunnel through to the swarm cluster to access our service, exposing the port 8080 on your local machine.

We can obtain the ip address of the swarm master node as follows.

In [ ]:
docker-machine ip swmaster

So from a terminal on your laptop open the ssh tunnel to **YOUR USER@YOUR SERVER**

MYSERVER=10.3.222.32
MYUSER=group20

e.g. ssh group20@10.3.222.32 -L 8080:192.168.99.114:8080 -Nv

In [ ]:
MYSERVER=10.3.222.32
MYUSER=group20
ssh ${MYUSER}@${MYSERVER} -L 8080:$(docker-machine ip swmaster):8080 -Nv

Then open your web browser at the page http://localhost:8080 and you should see a lovely blue whale, as below:

![](images/docker.png)


# scale service

Now we can scale the service to 3 replicas:

In [ ]:
docker $(docker-machine config swmaster) service scale docker-demo=3

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

# rolling-update

Now we will see how we can perform a rolling update.

We initially deployed version 20 of the service, now we will upgrade our whole cluster to version 20 


In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) service update --image mariolet/docker-demo:21 docker-demo

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

### Verifying the service has been updated

Then open your web browser at the page http://localhost:8080 and you should now see a lovely **red** whale.


![](images/docker_red.png)



# drain a node

We can drain a node effectively placing it in 'maintenance mode'.

Draining a node means that it no longer has running tasks on it.

In [ ]:
docker $(docker-machine config swmaster) node ls

Let's drain swnode1

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) node update --availability drain swnode1

and now we see that all services on swnode1 are shutdown

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

# remove a service

Now let's cleanup by removing our service

In [ ]:
docker $(docker-machine config swmaster) service rm docker-demo

We can check that the service is no longer running:

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) ps